In [5]:
import pandas as pd
from pathlib import Path
from utils import GetDatasetProfile
import os
import asyncio
import tqdm
root = '../sheets/mysql/'
files_names: list[str] = os.listdir(path=root)
files_names
tasks = [GetDatasetProfile(root, file_name = file) for file in files_names]
results = await asyncio.gather(*tasks)

Processing 13 columns (customer.csv)                                   ██████████████████████████████|  13/13  
Processing 8 columns (employee.csv)                                    ██████████████████████████████|   8/8   
Processing 9 columns (office.csv)                                      ██████████████████████████████|   9/9   
Processing 5 columns (orderdetails.csv)                                ██████████████████████████████|   5/5   
Processing 7 columns (orders.csv)                                      ██████████████████████████████|   7/7   
Processing 4 columns (payments.csv)                                    ██████████████████████████████|   4/4   
Processing 4 columns (productlines.csv)                                ██████████████████████████████|   4/4   
Processing 9 columns (products.csv)                                    ██████████████████████████████|   9/9   


In [14]:
result_dict = zip(files_names, results)
list(result_dict)[0][1]

'\n# Dataset Profile\nTotal columns: 13\n\n## Column Details\n| Column Name | Data Type | Sample Value | Stats | Additional Info |\n|------------|-----------|--------------|-------|------------------|\n| customerNumber | int64 | 103 | Total: 122, Nulls: 0 (0.0%), Unique: 122 | Mean: 296.4<br>Median: 298.5<br>Range: [103, 496] |\n| customerName | object | Atelier graphique | Total: 122, Nulls: 0 (0.0%), Unique: 122 | - |\n| contactLastName | object | Schmitt | Total: 122, Nulls: 0 (0.0%), Unique: 108 | - |\n| contactFirstName | object | Carine  | Total: 122, Nulls: 0 (0.0%), Unique: 111 | - |\n| phone | object | 40.32.2555 | Total: 122, Nulls: 0 (0.0%), Unique: 121 | - |\n| addressLine1 | object | 54, rue Royale | Total: 122, Nulls: 0 (0.0%), Unique: 122 | - |\n| addressLine2 | object | Level 3 | Total: 122, Nulls: 100 (82.0%), Unique: 21 | - |\n| city | object | Nantes | Total: 122, Nulls: 0 (0.0%), Unique: 96 | - |\n| state | object | NV | Total: 122, Nulls: 73 (59.8%), Unique: 18 | -

In [55]:
async def create(number: int, filename) -> int:
    id = number
    agent_name = f'agent_{id}_for_{filename}'
    print(f'creating file handling agent for file: {filename}')
    return agent_name

In [56]:
tasks = [create(number, filename=files_names[number]) for number in range(len(files_names))]

In [57]:
await asyncio.gather(*tasks)

creating file handling agent for file: customer.csv
creating file handling agent for file: employee.csv
creating file handling agent for file: office.csv
creating file handling agent for file: orderdetails.csv
creating file handling agent for file: orders.csv
creating file handling agent for file: payments.csv
creating file handling agent for file: productlines.csv
creating file handling agent for file: products.csv


['agent_0_for_customer.csv',
 'agent_1_for_employee.csv',
 'agent_2_for_office.csv',
 'agent_3_for_orderdetails.csv',
 'agent_4_for_orders.csv',
 'agent_5_for_payments.csv',
 'agent_6_for_productlines.csv',
 'agent_7_for_products.csv']